# The purpose of this project was to learn XGBoost classifier model and to understand its performance on predicting Customer churn rate

In [1]:
#Importing the required libraries
import pandas as pd
import numpy as np

#Importing plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#Importing statistical libraries
from sklearn.model_selection  import train_test_split
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb
import scikitplot as skplt

In [2]:
df = pd.read_excel('customer data.xlsx')
df.head()

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,...,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,...,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,...,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,86,5372,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,...,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,84,5003,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,...,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,89,5340,Competitor had better devices


In [3]:
df.tail()

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
7038,2569-WGERO,1,United States,California,Landers,92285,"34.341737, -116.539416",34.341737,-116.539416,Female,...,Two year,Yes,Bank transfer (automatic),21.15,1419.4,No,0,45,5306,NaN
7039,6840-RESVB,1,United States,California,Adelanto,92301,"34.667815, -117.536183",34.667815,-117.536183,Male,...,One year,Yes,Mailed check,84.80,1990.5,No,0,59,2140,NaN
7040,2234-XADUH,1,United States,California,Amboy,92304,"34.559882, -115.637164",34.559882,-115.637164,Female,...,One year,Yes,Credit card (automatic),103.20,7362.9,No,0,71,5560,NaN
7041,4801-JZAZL,1,United States,California,Angelus Oaks,92305,"34.1678, -116.86433",34.167800,-116.864330,Female,...,Month-to-month,Yes,Electronic check,29.60,346.45,No,0,59,2793,NaN
7042,3186-AJIEK,1,United States,California,Apple Valley,92308,"34.424926, -117.184503",34.424926,-117.184503,Male,...,Two year,Yes,Bank transfer (automatic),105.65,6844.5,No,0,38,5097,NaN


In [4]:
df.shape

(7043, 33)

In [5]:
df.columns

Index(['CustomerID', 'Count', 'Country', 'State', 'City', 'Zip Code',
       'Lat Long', 'Latitude', 'Longitude', 'Gender', 'Senior Citizen',
       'Partner', 'Dependents', 'Tenure Months', 'Phone Service',
       'Multiple Lines', 'Internet Service', 'Online Security',
       'Online Backup', 'Device Protection', 'Tech Support', 'Streaming TV',
       'Streaming Movies', 'Contract', 'Paperless Billing', 'Payment Method',
       'Monthly Charges', 'Total Charges', 'Churn Label', 'Churn Value',
       'Churn Score', 'CLTV', 'Churn Reason'],
      dtype='object')

In [6]:
df['Country'].unique()

array(['United States'], dtype=object)

In [7]:
df['State'].unique()

array(['California'], dtype=object)

In [8]:
df['City'].nunique()

1129

# Dropping the unnecessary columns

1. Customer ID, Count, Lat, Long are not useful for model development
2. Country and State only have a single value
3. The last 4 columns ie Churn Label, Churn Score, CLTV, Churn Reason contains information on exit interviews and not necessary for model development

In [9]:
df = df.drop(['CustomerID', 'Zip Code', 'Count', 'Country', 'State', 'Lat Long','Churn Label',
         'Churn Score', 'CLTV', 'Churn Reason'], axis=1)

In [10]:
df.columns

Index(['City', 'Latitude', 'Longitude', 'Gender', 'Senior Citizen', 'Partner',
       'Dependents', 'Tenure Months', 'Phone Service', 'Multiple Lines',
       'Internet Service', 'Online Security', 'Online Backup',
       'Device Protection', 'Tech Support', 'Streaming TV', 'Streaming Movies',
       'Contract', 'Paperless Billing', 'Payment Method', 'Monthly Charges',
       'Total Charges', 'Churn Value'],
      dtype='object')

In [11]:
df.columns.nunique()

23

In [12]:
df['City'].replace(" ", '_', regex=True, inplace=True)
df['City'].head()

0    Los_Angeles
1    Los_Angeles
2    Los_Angeles
3    Los_Angeles
4    Los_Angeles
Name: City, dtype: object

# Replacing the white spaces in the column names

In [13]:
df.columns = df.columns.str.replace(" ", "_")
df.columns

Index(['City', 'Latitude', 'Longitude', 'Gender', 'Senior_Citizen', 'Partner',
       'Dependents', 'Tenure_Months', 'Phone_Service', 'Multiple_Lines',
       'Internet_Service', 'Online_Security', 'Online_Backup',
       'Device_Protection', 'Tech_Support', 'Streaming_TV', 'Streaming_Movies',
       'Contract', 'Paperless_Billing', 'Payment_Method', 'Monthly_Charges',
       'Total_Charges', 'Churn_Value'],
      dtype='object')

In [14]:
df.isnull().sum()

City                 0
Latitude             0
Longitude            0
Gender               0
Senior_Citizen       0
Partner              0
Dependents           0
Tenure_Months        0
Phone_Service        0
Multiple_Lines       0
Internet_Service     0
Online_Security      0
Online_Backup        0
Device_Protection    0
Tech_Support         0
Streaming_TV         0
Streaming_Movies     0
Contract             0
Paperless_Billing    0
Payment_Method       0
Monthly_Charges      0
Total_Charges        0
Churn_Value          0
dtype: int64

Luckily, the dataframe does not have any missing values

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   City               7043 non-null   object 
 1   Latitude           7043 non-null   float64
 2   Longitude          7043 non-null   float64
 3   Gender             7043 non-null   object 
 4   Senior_Citizen     7043 non-null   object 
 5   Partner            7043 non-null   object 
 6   Dependents         7043 non-null   object 
 7   Tenure_Months      7043 non-null   int64  
 8   Phone_Service      7043 non-null   object 
 9   Multiple_Lines     7043 non-null   object 
 10  Internet_Service   7043 non-null   object 
 11  Online_Security    7043 non-null   object 
 12  Online_Backup      7043 non-null   object 
 13  Device_Protection  7043 non-null   object 
 14  Tech_Support       7043 non-null   object 
 15  Streaming_TV       7043 non-null   object 
 16  Streaming_Movies   7043 

The total charges column is under the data type string so checking for more information

In [16]:
df.loc[df['Total_Charges'] == " "]

,City,Latitude,Longitude,Gender,Senior_Citizen,Partner,Dependents,Tenure_Months,Phone_Service,Multiple_Lines,...,Device_Protection,Tech_Support,Streaming_TV,Streaming_Movies,Contract,Paperless_Billing,Payment_Method,Monthly_Charges,Total_Charges,Churn_Value
2234,San_Bernardino,34.084909,-117.258107,Female,No,Yes,No,0,No,No phone service,...,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,,0
2438,Independence,36.869584,-118.189241,Male,No,No,No,0,Yes,No,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,,0
2568,San_Mateo,37.590421,-122.306467,Female,No,Yes,No,0,Yes,No,...,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,,0
2667,Cupertino,37.306612,-122.080621,Male,No,Yes,Yes,0,Yes,Yes,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,,0
2856,Redcrest,40.363446,-123.835041,Female,No,Yes,No,0,No,No phone service,...,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,,0
4331,Los_Angeles,34.089953,-118.294824,Male,No,Yes,Yes,0,Yes,No,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,19.85,,0
4687,Sun_City,33.739412,-117.173334,Male,No,Yes,Yes,0,Yes,Yes,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,,0
5104,Ben_Lomond,37.078873,-122.090386,Female,No,Yes,Yes,0,Yes,No,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,,0
5719,La_Verne,34.144703,-117.770299,Male,No,Yes,Yes,0,Yes,No,...,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,,0
6772,Bell,33.970343,-118.171368,Female,No,Yes,Yes,0,Yes,Yes,...,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,,0


In [17]:
print(len(df.loc[df['Total_Charges'] == " "]))

11


There are 11 observations where the total charges are having blank information and tenure in months is also zero for these observations so lets replace the information to 0

In [18]:
df.loc[df['Total_Charges'] == " ", 'Total_Charges'] = 0

In [19]:
print(len(df.loc[df['Total_Charges'] == " "]))

0


In [20]:
df['Total_Charges'] = df['Total_Charges'].astype(float)
df['Total_Charges'].dtypes

dtype('float64')

In [21]:
#Replacing all the white spaces in the df with an underscore (_)

df.replace(" ",'_', regex=True, inplace=True)
df.head()

,City,Latitude,Longitude,Gender,Senior_Citizen,Partner,Dependents,Tenure_Months,Phone_Service,Multiple_Lines,...,Device_Protection,Tech_Support,Streaming_TV,Streaming_Movies,Contract,Paperless_Billing,Payment_Method,Monthly_Charges,Total_Charges,Churn_Value
0,Los_Angeles,33.964131,-118.272783,Male,No,No,No,2,Yes,No,...,No,No,No,No,Month-to-month,Yes,Mailed_check,53.85,108.15,1
1,Los_Angeles,34.059281,-118.307420,Female,No,No,Yes,2,Yes,No,...,No,No,No,No,Month-to-month,Yes,Electronic_check,70.70,151.65,1
2,Los_Angeles,34.048013,-118.293953,Female,No,No,Yes,8,Yes,Yes,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic_check,99.65,820.50,1
3,Los_Angeles,34.062125,-118.315709,Female,No,Yes,Yes,28,Yes,Yes,...,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic_check,104.80,3046.05,1
4,Los_Angeles,34.039224,-118.266293,Male,No,No,Yes,49,Yes,Yes,...,Yes,No,Yes,Yes,Month-to-month,Yes,Bank_transfer_(automatic),103.70,5036.30,1


# Splitting data based on Target variable and Features

In [22]:
X = df.drop(['Churn_Value'], axis=1).copy()
X.head()

,City,Latitude,Longitude,Gender,Senior_Citizen,Partner,Dependents,Tenure_Months,Phone_Service,Multiple_Lines,...,Online_Backup,Device_Protection,Tech_Support,Streaming_TV,Streaming_Movies,Contract,Paperless_Billing,Payment_Method,Monthly_Charges,Total_Charges
0,Los_Angeles,33.964131,-118.272783,Male,No,No,No,2,Yes,No,...,Yes,No,No,No,No,Month-to-month,Yes,Mailed_check,53.85,108.15
1,Los_Angeles,34.059281,-118.307420,Female,No,No,Yes,2,Yes,No,...,No,No,No,No,No,Month-to-month,Yes,Electronic_check,70.70,151.65
2,Los_Angeles,34.048013,-118.293953,Female,No,No,Yes,8,Yes,Yes,...,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic_check,99.65,820.50
3,Los_Angeles,34.062125,-118.315709,Female,No,Yes,Yes,28,Yes,Yes,...,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic_check,104.80,3046.05
4,Los_Angeles,34.039224,-118.266293,Male,No,No,Yes,49,Yes,Yes,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank_transfer_(automatic),103.70,5036.30


In [23]:
y = df['Churn_Value'].copy()
y.head()

0    1
1    1
2    1
3    1
4    1
Name: Churn_Value, dtype: int64

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   City               7043 non-null   object 
 1   Latitude           7043 non-null   float64
 2   Longitude          7043 non-null   float64
 3   Gender             7043 non-null   object 
 4   Senior_Citizen     7043 non-null   object 
 5   Partner            7043 non-null   object 
 6   Dependents         7043 non-null   object 
 7   Tenure_Months      7043 non-null   int64  
 8   Phone_Service      7043 non-null   object 
 9   Multiple_Lines     7043 non-null   object 
 10  Internet_Service   7043 non-null   object 
 11  Online_Security    7043 non-null   object 
 12  Online_Backup      7043 non-null   object 
 13  Device_Protection  7043 non-null   object 
 14  Tech_Support       7043 non-null   object 
 15  Streaming_TV       7043 non-null   object 
 16  Streaming_Movies   7043 

# One hot encoding of categorical variables

In [25]:
X_encoded = pd.get_dummies(X, columns=['City', 'Gender', 'Senior_Citizen', 'Partner', 'Dependents', 'Phone_Service',
                           'Multiple_Lines', 'Internet_Service', 'Online_Security', 
                           'Online_Backup', 'Device_Protection','Tech_Support', 'Streaming_TV','Streaming_Movies',
                           'Contract','Paperless_Billing','Payment_Method'])

In [26]:
X_encoded.head()

,Latitude,Longitude,Tenure_Months,Monthly_Charges,Total_Charges,City_Acampo,City_Acton,City_Adelanto,City_Adin,City_Agoura_Hills,...,Streaming_Movies_Yes,Contract_Month-to-month,Contract_One_year,Contract_Two_year,Paperless_Billing_No,Paperless_Billing_Yes,Payment_Method_Bank_transfer_(automatic),Payment_Method_Credit_card_(automatic),Payment_Method_Electronic_check,Payment_Method_Mailed_check
0,33.964131,-118.272783,2,53.85,108.15,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
1,34.059281,-118.307420,2,70.70,151.65,0,0,0,0,0,...,0,1,0,0,0,1,0,0,1,0
2,34.048013,-118.293953,8,99.65,820.50,0,0,0,0,0,...,1,1,0,0,0,1,0,0,1,0
3,34.062125,-118.315709,28,104.80,3046.05,0,0,0,0,0,...,1,1,0,0,0,1,0,0,1,0
4,34.039224,-118.266293,49,103.70,5036.30,0,0,0,0,0,...,1,1,0,0,0,1,1,0,0,0


In [27]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, random_state=42, stratify=y)

In [28]:
print(y_train.sum()/len(y_train))

0.2654297614539947


In [29]:
print(y_test.sum()/len(y_test))

0.26519023282226006


In [30]:
print(X_train.shape)

(5282, 1177)


In [31]:
print(X_test.shape)

(1761, 1177)


# XG Boost Model

In [32]:
xgb_model = xgb.XGBClassifier(objective='binary:logistic', missing=None, seed=42)
xgb_model.fit(X_train, y_train,
              verbose=True,
              early_stopping_rounds=10,
              eval_metric='aucpr',
              eval_set=[(X_test, y_test)])

/Users/rajeevr/anaconda3/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/Users/rajeevr/anaconda3/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-aucpr:0.63387
[1]	validation_0-aucpr:0.64922
[2]	validation_0-aucpr:0.65710
[3]	validation_0-aucpr:0.65416
[4]	validation_0-aucpr:0.64873
[5]	validation_0-aucpr:0.65800
[6]	validation_0-aucpr:0.65458
[7]	validation_0-aucpr:0.65233
[8]	validation_0-aucpr:0.65510
[9]	validation_0-aucpr:0.65857
[10]	validation_0-aucpr:0.65604
[11]	validation_0-aucpr:0.65847
[12]	validation_0-aucpr:0.65757
[13]	validation_0-aucpr:0.65931
[14]	validation_0-aucpr:0.65931
[15]	validation_0-aucpr:0.65847
[16]	validation_0-aucpr:0.65525
[17]	validation_0-aucpr:0.65373
[18]	validation_0-aucpr:0.65280
[19]	validation_0-aucpr:0.65491
[20]	validation_0-aucpr:0.65442
[21]	validation_0-aucpr:0.65451
[22]	validation_0-aucpr:0.65244
[23]	validation_0-aucpr:0.65383


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=None, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

As we can see that the aucpr has stopped at 0.65383 indicating a decent model not really applicable in the real world. 